# Energy Landscape & ΔH Convergence

Track total energy (ΔH) under uniform vs diffusion-gated query influence. We approximate an iterative trajectory by recording ΔH after each partial settle call with a decreasing tolerance schedule.

Sections:
1. Embeddings (synthetic semantic clusters)
2. Uniform lattice pseudo-iteration trajectory
3. Diffusion gating trajectory
4. Overlay plot & summary stats
5. Energy reduction and gate distribution


In [ ]:
# 1. Imports & synthetic clustered embeddings
import matplotlib.pyplot as plt
import numpy as np

from oscillink import OscillinkLattice, compute_diffusion_gates

np.random.seed(7)

# Build synthetic embeddings: 3 clusters with slight offsets
clusters = 3
points_per = 80
D = 64
base = np.random.randn(clusters, D).astype(np.float32)
base /= np.linalg.norm(base, axis=1, keepdims=True)
Y_list = []
labels = []
for c in range(clusters):
    center = base[c]
    for _ in range(points_per):
        vec = center + 0.25 * np.random.randn(D).astype(np.float32)
        vec /= (np.linalg.norm(vec) + 1e-12)
        Y_list.append(vec)
        labels.append(c)
Y = np.vstack(Y_list).astype(np.float32)
N = Y.shape[0]
print("Embeddings shape:", Y.shape)

# Query as mixture of cluster 0 and 1
psi = (base[0] + 0.8 * base[1])
psi /= (np.linalg.norm(psi) + 1e-12)


In [ ]:
# 2. Helper to record pseudo-iterative ΔH by repeated short settles

def energy_trajectory(lattice: OscillinkLattice, steps: int = 10, base_tol: float = 1e-2):
    traj = []
    for s in range(steps):
        # progressively tighten tolerance; limit iterations to simulate incremental refinement
        tol = base_tol * (0.5 ** s)
        lattice.settle(max_iters=4, tol=tol)
        rec = lattice.receipt()
        traj.append(rec["deltaH_total"])
    return traj

# Uniform lattice
lat_uniform = OscillinkLattice(Y, kneighbors=8, lamG=1.0, lamC=0.6, lamQ=3.5, deterministic_k=True)
lat_uniform.set_query(psi)
traj_uniform = energy_trajectory(lat_uniform, steps=9)
print("Final uniform ΔH:", traj_uniform[-1])

In [ ]:
# 3. Diffusion gating trajectory
gates = compute_diffusion_gates(Y, psi, kneighbors=8, beta=1.0, gamma=0.12, deterministic_k=True)
lat_diff = OscillinkLattice(Y, kneighbors=8, lamG=1.0, lamC=0.6, lamQ=3.5, deterministic_k=True)
lat_diff.set_query(psi, gates=gates)
traj_diff = energy_trajectory(lat_diff, steps=9)
print("Final diffusion ΔH:", traj_diff[-1])
print("Energy reduction %:", 100.0 * (traj_uniform[-1] - traj_diff[-1]) / traj_uniform[-1])

In [ ]:
# 4. Overlay plot
plt.figure(figsize=(6,4))
plt.plot(traj_uniform, marker='o', label='Uniform')
plt.plot(traj_diff, marker='o', label='Diffusion')
plt.xlabel('Pseudo-iteration')
plt.ylabel('ΔH total')
plt.title('Energy Trajectory (Lower is Better)')
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# 5. Gate distribution summary
print(f"Gate stats: min={gates.min():.3f} max={gates.max():.3f} mean={gates.mean():.3f}")
print("Head gates:", np.round(gates[:10], 3))